### Methods proposed in the article 

In [1]:
import numpy as np
import os
import sys
project_dir = os.getcwd().split('notebooks')[0]
sys.path.append(project_dir)

In [2]:
from src.helper import *

#### 1) Algorithm 1: NaiveMapper

In [3]:
def NaiveMapper(A):
    """
    Function that computes sparse matrix multiplication
    input: 
    A: sparse matrix
    output:
    AProd: dot products between columns of A
    """
    (n,m) = A.shape
    AProd = sum( [A[i][0]*A[i] for i in range(len(A))] ) # we walk through the rows
    for r in range(1,m): # then emit the columns
        AProd = np.vstack((AProd,sum( [A[i][r]*A[i] for i in range(len(A))] ) ))
    return AProd
    

#### 2) Algorithm 2: NaiveReducer

In [4]:
def NaiveReducer(C):
    """
    Function that computes the sum of lists of arrays
    input:
    C: list of arrays
    output:
    sum(C): sum of list of arrays
    """
    return sum(C)

In [17]:
# Example
A = sparse_generator(3,4)
NaiveMapper(A)

#A^T A:
NaiveReducer(NaiveMapper(A))

array([0.29507559, 0.90107394, 0.        , 0.        ])

#### 3) Algorithm 3: DIMSUMMapper

In [18]:
def norm(A):
    """
    Function that computes the norm 2 of each column of a matrix
    input:
    A: matrix
    output:
    norm: array of norms of each column
    """
    norm = (np.square(A).sum(axis=0))**(1/2) 
    return norm

In [19]:
(n,m) = A.shape

In [21]:
A

array([[ 0.54320861,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.0648094 ,  0.        ,  0.        ],
       [ 0.        , -0.94703415,  0.        ,  0.        ]])

In [ ]:
def DIMSUMMApper(A, gamma):
    (n,m) = A.shape 
    Anorm = norm(A) # norm of columns of A
    for ci in range(n):
        for cj in range(n):
            